In [77]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from mpl_toolkits.mplot3d import axes3d
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import cm
import random
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import FloatSlider
import ipywidgets as widgets
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='alanTorresManci', api_key='6DR9Y3FeiXneNwynb5W4')
%matplotlib inline

In [78]:
def interp2d_interleave(z,n):
    '''performs linear interpolation on a grid

    all points are interpolated in one step not recursively

    Parameters
    ----------
    z : 2d array (M,N)
    n : int
        number of points interpolated

    Returns
    -------
    zi : 2d array ((M-1)*n+M, (N-1)*n+N)
        original and linear interpolated values

    '''
    frac = np.atleast_2d(np.arange(0,n+1)/(1.0+n)).T
    zi1 = np.kron(z[:,:-1],np.ones(len(frac))) + np.kron(np.diff(z),frac.T)
    zi1 = np.hstack((zi1,z[:,-1:]))
    zi2 = np.kron(zi1.T[:,:-1],np.ones(len(frac))) + np.kron(np.diff(zi1.T),frac.T)
    zi2 = np.hstack((zi2,zi1.T[:,-1:]))
    return zi2.T

def interp2d_interleave_recursive(z,n):
    '''interpolates by recursively interleaving n times
    '''
    zi = z.copy()
    for ii in range(1,n+1):
        zi = interp2d_interleave(zi,1)
    return zi

def indices(a, func):
    return [i for (i, val) in enumerate(a) if func(val)]


In [79]:
terrain = np.zeros((2, 2))
randmap = 0.1 * np.random.randn(len(terrain[0, :]), len(terrain[0, :]));
terrain
#np.ones(terrain.shape())

array([[0., 0.],
       [0., 0.]])

In [80]:
def generateGround(i, c):
    terrain = np.zeros((2, 2))
    for j in range(i):
        terrain = interp2d_interleave_recursive(terrain, 1)
        subsquares = np.ones(terrain.shape)
        for k in range(len(terrain[0, :])):
            if k%2 == 0:
                subsquares[k, k] = 0
        randmap = c * np.random.randn(len(terrain[0, :]), len(terrain[0, :]));
        for l in range(len(terrain[0, :])):
            for n in range(len(terrain[0, :])):
                if(subsquares[l, n] != 0):
                    terrain[l, n] = terrain[l, n] + randmap[l, n]
        c = c/2
    return terrain

In [81]:

terreno.shape

(257, 257)

In [91]:
z_data = generateGround(8, 0.1)
data = [
    go.Surface(
        z=z_data,
        contours=go.surface.Contours(
            z=go.surface.contours.Z(
              show=True,
              usecolormap=True,
              highlightcolor="#42f462",
              project=dict(z=True)
            )
        )
    )
]
layout = go.Layout(
    title='Random Mountain Surface',
    autosize=False,
    scene=dict(camera=dict(eye=dict(x=1.87, y=0.88, z=-0.64))),
    width=500,
    height=500,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='elevations-3d-surface-contours')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~alanTorresManci/0 or inside your plot.ly account where it is named 'elevations-3d-surface-contours'


In [87]:
def generador(i):
    terreno = generateGround(i, 0.1)
    z = terreno
    x, y = np.linspace(0,len(z[0,:]), len(z[0,:])), np.linspace(0, len(z[0,:]), len(z[0,:]))
    X, Y = np.meshgrid(x, y)
    fig = plt.figure(figsize=(8,6))
    ax = fig.gca(projection='3d')
    #y = x
    #xx = np.array(np.meshgrid(x, y))
    #yy = xx



    ax.plot_surface(X, Y, abs(z), cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)
    plt.show()

In [88]:
interact(generador, i = widgets.IntSlider(min = 0, max = 9, value = 0))

interactive(children=(IntSlider(value=0, description='i', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.generador>